<a href="https://colab.research.google.com/github/akshatshaw/Case-Study/blob/main/src/gpt_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/akshatshaw/Case-Study

In [2]:
cd Case-Study/

[WinError 2] The system cannot find the file specified: 'Case-Study/'
c:\Users\AKSHAT SHAW\OneDrive - iitr.ac.in\Desktop\SEM 6\case-study\src


In [3]:
import numpy as np
import pandas as pd
import torch

c:\Users\AKSHAT SHAW\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
data = pd.read_csv(r"C:\Users\AKSHAT SHAW\OneDrive - iitr.ac.in\Desktop\SEM 6\case-study\Code\pit_cleaned_final.csv")
data.head()

,Fe,Cr,Ni,Mo,W,N,Nb,C,Si,Mn,...,Test Solution,[Cl-] M,pH,Test Method,Scan Rate mV/s,Heat treatment,Microstructures,Comments,Material class,combine_text
0,69.71,18.0,10.0,0.0,0.0,0.0,0.0,0.03,1.0,1.0,...,"300 ppm NaCl, pH 7.8",0.0051,7.8,Potentiodynamic Polarization with exposed area...,0.1,Not Available,Not Available,S30403 (304L),1,"<S> Test Solution: 0 300 ppm NaCl, pH 7...."
1,69.71,18.0,10.0,0.0,0.0,0.0,0.0,0.03,1.0,1.0,...,"500 ppm NaCl, pH 7.8",0.0086,7.8,Potentiodynamic Polarization with exposed area...,0.1,Not Available,Not Available,S30403 (304L),1,"<S> Test Solution: 0 300 ppm NaCl, pH 7...."
2,69.71,18.0,10.0,0.0,0.0,0.0,0.0,0.03,1.0,1.0,...,"1000 ppm NaCl, pH 7.8",0.0171,7.8,Potentiodynamic Polarization with exposed area...,0.1,Not Available,Not Available,S30403 (304L),1,"<S> Test Solution: 0 300 ppm NaCl, pH 7...."
3,69.71,18.0,10.0,0.0,0.0,0.0,0.0,0.03,1.0,1.0,...,"5000 ppm NaCl, pH 7.8",0.0856,7.8,Potentiodynamic Polarization with exposed area...,0.1,Not Available,Not Available,S30403 (304L),1,"<S> Test Solution: 0 300 ppm NaCl, pH 7...."
4,69.71,18.0,10.0,0.0,0.0,0.0,0.0,0.03,1.0,1.0,...,Seawater Natural,0.5460,8.2,Potentiodynamic Polarization with exposed area...,0.1,Not Available,Not Available,S30403 (304L),1,"<S> Test Solution: 0 300 ppm NaCl, pH 7...."


In [5]:
num = data.select_dtypes(include="number").columns
cat = data.select_dtypes(exclude="number").columns
num, cat

(Index(['Fe', 'Cr', 'Ni', 'Mo', 'W', 'N', 'Nb', 'C', 'Si', 'Mn', 'Cu', 'P', 'S',
        'Al', 'V', 'Ta', 'Re', 'Ce', 'Ti', 'Co', 'B', 'Mg', 'Y', 'Gd',
        'Epit, mV (SCE)', 'Test Temp (C)', '[Cl-] M', 'pH', 'Scan Rate mV/s',
        'Material class'],
       dtype='object'),
 Index(['Test Solution', 'Test Method', 'Heat treatment', 'Microstructures',
        'Comments', 'combine_text'],
       dtype='object'))

In [6]:
num = ['Fe', 'Cr', 'Ni', 'Mo', 'W', 'N', 'Nb', 'C', 'Si', 'Mn', 'Cu', 'P', 'S',
        'Al', 'V', 'Ta', 'Re', 'Ce', 'Ti', 'Co', 'B', 'Mg', 'Y', 'Gd',
         'Test Temp (C)', '[Cl-] M', 'pH', 'Scan Rate mV/s',
        'Material class']
num_target = ['Epit, mV (SCE)']

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[num] = scaler.fit_transform(data[num])

In [8]:
# Creating the data loaders
from sklearn.model_selection import train_test_split
train, val = train_test_split(data, test_size=0.1, random_state=42)


# Print the sizes
print(f"Train set: {len(train)} rows")
print(f"Validation set: {len(val)} rows")


Train set: 640 rows
Validation set: 72 rows


### Setting the dataloader for the text data

In [9]:
cd src

[WinError 2] The system cannot find the file specified: 'src'
c:\Users\AKSHAT SHAW\OneDrive - iitr.ac.in\Desktop\SEM 6\case-study\src


In [10]:
from gpt import *
import torch
from torch.utils.data import Dataset

class Dataset_txt(Dataset):

    def __init__(self, data, tokenizer,txt_col, target = None, max_length=None, pad_token_id=50256):
        self.data = data
        self.target = target
        self.txt_col = txt_col
        self.encoded_texts = [tokenizer.encode(text) for text in self.data[f"{txt_col}"]]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length

        self.encoded_texts = [
            encoded_text[:self.max_length] for encoded_text in self.encoded_texts
        ]

        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        # label = self.data.iloc[index][f"{self.target}"]
        return (
            torch.tensor(encoded, dtype=torch.long)
            # torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length


In [11]:
# !pip install -q tiktoken
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [12]:
train_data = Dataset_txt(
    data = train,
    tokenizer=tokenizer,
    txt_col="combine_text",
    max_length=1024
)
val_data = Dataset_txt(
    data = val,
    tokenizer=tokenizer,
    txt_col="combine_text",
    max_length=1024
)
len(train_data)

640

In [13]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)
val_loader = DataLoader(
    dataset=val_data,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [14]:
class Dataset_Num(Dataset):
    def __init__(self, data, numerical_features):
        self.data = data
        self.numerical_features = numerical_features
    def __getitem__(self, index):
        return (
           torch.tensor(self.data.iloc[index].loc[self.numerical_features], dtype=torch.float),
            torch.tensor(self.data.iloc[index].loc["Epit, mV (SCE)"], dtype=torch.float)
        )
    def __len__(self):
        return len(self.data)

In [15]:
train_num = Dataset_Num(
    data = train,
    numerical_features = num
)
val_num = Dataset_Num(
    data = val,
    numerical_features = num
)


In [16]:
train_num_loader = DataLoader(
    dataset=train_num,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)
val_num_loader = DataLoader(
    dataset=val_num,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)


In [17]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [18]:
print("Train loader:")
for x, y in train_num_loader:
 print(x.shape, y.shape)
 break
print("\nValidation loader:")
for x, y in val_num_loader:
 print(x.shape, y.shape)
 break

Train loader:
torch.Size([8, 29]) torch.Size([8])

Validation loader:
torch.Size([8, 29]) torch.Size([8])


In [19]:
print("Train loader:")
for x in train_loader:
 print(x.shape)
 break
print("\nValidation loader:")
for x in val_loader:
 print(x.shape)
 break

Train loader:
torch.Size([8, 1024])

Validation loader:
torch.Size([8, 1024])


In [20]:
# loading the orignal model
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"
BASE_CONFIG = {
 "vocab_size": 50257,
 "context_length": 1024,
 "drop_rate": 0.0,
 "qkv_bias": True
}
model_configs = {
 "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
 "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
 "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
 "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

GPT_CONFIG_124M = {
 "vocab_size": 50257,
 "context_length": 256, # We shorten the context length from 1,024 to 256 tokens. Original GPT-2 has a context length of 1,024 tokens.
 "emb_dim": 768,
 "n_heads": 12,
 "n_layers": 12,
 "drop_rate": 0.1,
 "qkv_bias": False
}


In [21]:
# from gpt_download import download_and_load_gpt2, load_weights_into_gpt
# from gpt import GPTModel
# model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")

# settings, params = download_and_load_gpt2(
#         model_size=model_size, models_dir="gpt2"
# )

# gpt_model = GPTModel(BASE_CONFIG)
# load_weights_into_gpt(gpt_model, params)

In [22]:
# gpt_model.eval()
# for param in gpt_model.parameters():
#     param.requires_grad = False

In [23]:
# torch.manual_seed(123)

# num_classes = 32 # we take a 32dim output form the gpt model for the representation of text.

# gpt_model.out_head = torch.nn.Linear(
#  in_features=BASE_CONFIG["emb_dim"],
#  out_features=num_classes
# )

In [24]:
# # For training the final layer norm and last transformer block trainable
# for param in gpt_model.trf_blocks[-1].parameters():
#  param.requires_grad = True
# for param in gpt_model.final_norm.parameters():
#  param.requires_grad = True


In [25]:
class PitModel(torch.nn.Module):
    def __init__(self,  embedding_dim = 32, numerical_features= len(num)): #gpt_model,
        super().__init__()
        self.num_classes = embedding_dim + numerical_features
        
        # self.gpt_model = gpt_model
        self.tok_emb = nn.Embedding(50257, 256) #vocab size & emb_dim
        self.pos_emb = nn.Embedding(1024, 256) #context_length & emb_dim
        self.lstm = nn.LSTM(
            input_size=256,
            hidden_size=32,
            batch_first=True,
        )
        
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(self.num_classes, 256),
            torch.nn.Dropout(0.3),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 32),
            torch.nn.Dropout(0.5),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x_num, x_text):
        # x = self.gpt_model(x_text)
        # x = x[:, -1, :]
        # x = torch.cat((x, x_num), dim=1)
        
        with torch.no_grad():  # Ensure no gradients are calculated for text processing
            tok_embeds = self.tok_emb(x_text)
            pos_embeds = self.pos_emb(
                torch.arange(1024, device=x_text.device)
            )
            x = tok_embeds + pos_embeds
            x = self.lstm(x)[0]
            x= x[:, -1, :]
        
        # Text features are now detached from computation graph
        x = x.detach()
        
        x = torch.cat((x, x_num), dim=1)
        x = self.linear(x)
        return x

In [26]:
model = PitModel()

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PitModel(
  (tok_emb): Embedding(50257, 256)
  (pos_emb): Embedding(1024, 256)
  (lstm): LSTM(256, 32, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=61, out_features=256, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=32, bias=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [28]:
import torch
import torch.nn.functional as F
import numpy as np

def calc_loss_batch(num_batch, txt_batch, target_batch, model, device):
    num_batch = num_batch.to(device)
    txt_batch = txt_batch.to(device)
    target_batch = target_batch.float().to(device)

    # Forward pass through the model with both numerical and text inputs
    predictions = model(num_batch, txt_batch)

    # Ensure predictions and targets have the same shape
    if predictions.shape != target_batch.shape:
        predictions = predictions.squeeze()

    # Calculate both MAE and MSE losses
    mae_loss = F.l1_loss(predictions, target_batch)
    mse_loss = F.mse_loss(predictions, target_batch)

    return {'mae': mae_loss, 'mse': mse_loss, 'predictions': predictions, 'targets': target_batch}

def calc_loss_loader(data_num_loader, data_txt_loader, model, device, num_batches=None):
    total_mae = 0.
    total_mse = 0.
    all_predictions = []
    all_targets = []

    # Check if both loaders have data
    if len(data_num_loader) == 0 or len(data_txt_loader) == 0:
        return {'mae': float("nan"), 'rmse': float("nan"), 'r2': float("nan")}

    # Determine number of batches to process
    if num_batches is None:
        num_batches = min(len(data_num_loader), len(data_txt_loader))
    else:
        num_batches = min(num_batches, min(len(data_num_loader), len(data_txt_loader)))

    # Create iterators for both loaders
    num_iterator = iter(data_num_loader)
    txt_iterator = iter(data_txt_loader)

    for i in range(num_batches):
        try:
            # Get batches from both loaders
            num_batch, target_batch = next(num_iterator)
            txt_batch = next(txt_iterator)  # Ignore targets from text loader

            batch_results = calc_loss_batch(num_batch, txt_batch, target_batch, model, device)

            total_mae += batch_results['mae'].item()
            total_mse += batch_results['mse'].item()

            # Collect predictions and targets for R² calculation
            all_predictions.append(batch_results['predictions'].cpu().detach())
            all_targets.append(batch_results['targets'].cpu().detach())

        except StopIteration:
            # Handle case where one loader is exhausted before the other
            break

    # Calculate final metrics
    num_processed_batches = len(all_predictions)
    if num_processed_batches == 0:
        return {'mae': float("nan"), 'rmse': float("nan"), 'r2': float("nan")}

    mae = total_mae / num_processed_batches
    rmse = (total_mse / num_processed_batches) ** 0.5

    # Calculate R² score
    all_predictions = torch.cat(all_predictions).numpy()
    all_targets = torch.cat(all_targets).numpy()

    # R² = 1 - (sum of squared residuals / total sum of squares)
    ss_res = ((all_targets - all_predictions) ** 2).sum()
    ss_tot = ((all_targets - all_targets.mean()) ** 2).sum()

    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0

    return {'mae': mae, 'rmse': rmse, 'r2': r2}


In [29]:
with torch.no_grad():
    train_loss = calc_loss_loader(train_num_loader, train_loader, model, device, num_batches=8)
    test_loss = calc_loss_loader(train_num_loader, val_loader, model, device, num_batches=8)
train_loss, test_loss

({'mae': 392.8625602722168,
  'rmse': 483.4590997921903,
  'r2': -0.4648319482803345},
 {'mae': 402.61352157592773,
  'rmse': 494.5374871249651,
  'r2': -0.7143212556838989})

In [30]:
import os
from dotenv import load_dotenv, dotenv_values 
load_dotenv() 

import wandb
wandb.login(key = os.getenv("WANDB_API_KEY"))

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: akshatshaw47 (akshatshaw47-iit-roorkee). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\AKSHAT SHAW\_netrc


True

In [31]:
def trainer(
    model, train_num_loader, train_txt_loader, val_num_loader, val_txt_loader, optimizer, device,
    num_epochs, eval_freq, eval_iter, loss_function="mae",
    project_name=None, run_name=None):

    # Initialize wandb
    wandb.init(project=project_name, name=run_name)

    # Log hyperparameters
    wandb.config.update({
        "epochs": num_epochs,
        "eval_frequency": eval_freq,
        "eval_iterations": eval_iter,
        "optimizer": optimizer.__class__.__name__,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "device": device,
        "model_name": model.__class__.__name__
    })

    train_losses, val_losses = [], []
    train_maes, val_maes = [], []
    train_rmses, val_rmses = [], []
    train_r2s, val_r2s = [], []
    examples_seen, global_step = 0, -1

    # Create iterators for the training loaders
    num_train_iter = iter(train_num_loader)
    txt_train_iter = iter(train_txt_loader)

    for epoch in range(num_epochs):
        model.train()
        epoch_mae_loss = 0
        epoch_mse_loss = 0
        batch_count = 0

        # Reset iterators at the beginning of each epoch
        num_train_iter = iter(train_num_loader)
        txt_train_iter = iter(train_txt_loader)

        # Determine number of batches for this epoch
        num_batches = min(len(train_num_loader), len(train_txt_loader))

        for _ in range(num_batches):
            try:
                # Get batches from both loaders
                num_batch, target_batch = next(num_train_iter)
                txt_batch = next(txt_train_iter)  # Ignore targets from text loader

                optimizer.zero_grad()
                loss_dict = calc_loss_batch(
                    num_batch, txt_batch, target_batch, model, device
                )
                # For backward pass, you can choose either MAE or MSE or a combination
                loss = loss_dict[f'{loss_function}']
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                # Track batch-level metrics
                epoch_mae_loss += loss_dict['mae'].item()
                epoch_mse_loss += loss_dict['mse'].item()
                batch_count += 1
                examples_seen += num_batch.shape[0]
                global_step += 1

                # Log batch metrics
                wandb.log({
                    "batch_mae": loss_dict['mae'].item(),
                    "batch_mse": loss_dict['mse'].item(),
                    "batch_rmse": loss_dict['mse'].item() ** 0.5,
                    "examples_seen": examples_seen,
                    "global_step": global_step
                }, step=global_step)

                if global_step % eval_freq == 0:
                    # Evaluate model
                    train_metrics = calc_loss_loader(train_num_loader, train_txt_loader, model, device, eval_iter)
                    val_metrics = calc_loss_loader(val_num_loader, val_txt_loader, model, device, eval_iter)

                    # Store metrics
                    train_losses.append(train_metrics['mae'])
                    val_losses.append(val_metrics['mae'])
                    train_maes.append(train_metrics['mae'])
                    val_maes.append(val_metrics['mae'])
                    train_rmses.append(train_metrics['rmse'])
                    val_rmses.append(val_metrics['rmse'])
                    train_r2s.append(train_metrics['r2'])
                    val_r2s.append(val_metrics['r2'])

                    # Log evaluation metrics
                    wandb.log({
                        "train_mae": train_metrics['mae'],
                        "val_mae": val_metrics['mae'],
                        "train_rmse": train_metrics['rmse'],
                        "val_rmse": val_metrics['rmse'],
                        "train_r2": train_metrics['r2'],
                        "val_r2": val_metrics['r2'],
                        "epoch": epoch + 1
                    }, step=global_step)

                    print(f"Ep {epoch+1} (Step {global_step:06d}): "
                          f"Train MAE {train_metrics['mae']:.3f}, "
                          f"Val MAE {val_metrics['mae']:.3f}, "
                          f"Train R² {train_metrics['r2']:.3f}, "
                          f"Val R² {val_metrics['r2']:.3f}")

            except StopIteration:
                # Handle case where one loader is exhausted before the other
                break

        # Calculate and log metrics at epoch end
        train_metrics = calc_loss_loader(train_num_loader, train_txt_loader, model, device, eval_iter)
        val_metrics = calc_loss_loader(val_num_loader, val_txt_loader, model, device, eval_iter)

        # Log epoch-level metrics
        wandb.log({
            "epoch": epoch + 1,
            "epoch_avg_mae": epoch_mae_loss / batch_count if batch_count > 0 else float('nan'),
            "epoch_avg_mse": epoch_mse_loss / batch_count if batch_count > 0 else float('nan'),
            "epoch_avg_rmse": (epoch_mse_loss / batch_count) ** 0.5 if batch_count > 0 else float('nan'),
            "train_mae": train_metrics['mae'],
            "val_mae": val_metrics['mae'],
            "train_rmse": train_metrics['rmse'],
            "val_rmse": val_metrics['rmse'],
            "train_r2": train_metrics['r2'],
            "val_r2": val_metrics['r2'],
            "learning_rate": optimizer.param_groups[0]['lr']
        }, step=global_step)

        print(f"Training MAE: {train_metrics['mae']:.4f} | ", end="")
        print(f"Validation MAE: {val_metrics['mae']:.4f} | ", end="")
        print(f"Training RMSE: {train_metrics['rmse']:.4f} | ", end="")
        print(f"Validation RMSE: {val_metrics['rmse']:.4f} | ", end="")
        print(f"Training R²: {train_metrics['r2']:.4f} | ", end="")
        print(f"Validation R²: {val_metrics['r2']:.4f}")

    # Close wandb run
    wandb.finish()

    results = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_maes': train_maes,
        'val_maes': val_maes,
        'train_rmses': train_rmses,
        'val_rmses': val_rmses,
        'train_r2s': train_r2s,
        'val_r2s': val_r2s,
        'examples_seen': examples_seen
    }

    return results


In [32]:
import time
start_time = time.time()
torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=5, verbose=True
# )
results = trainer(
    model,train_num_loader, train_loader, val_num_loader, val_loader, optimizer, device,
    num_epochs=5000, eval_freq=10, eval_iter=10,
    project_name="gpt2-corr-txt-num", run_name="lstm-5-no-txt-grad"
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train MAE 434.637, Val MAE 369.701, Train R² -0.261, Val R² -0.559
Ep 1 (Step 000010): Train MAE 421.412, Val MAE 369.698, Train R² -0.787, Val R² -0.559
Ep 1 (Step 000020): Train MAE 432.945, Val MAE 369.692, Train R² -0.866, Val R² -0.559
Ep 1 (Step 000030): Train MAE 399.863, Val MAE 369.681, Train R² -0.634, Val R² -0.559
Ep 1 (Step 000040): Train MAE 376.886, Val MAE 369.673, Train R² -0.206, Val R² -0.559
Ep 1 (Step 000050): Train MAE 462.819, Val MAE 369.668, Train R² -0.465, Val R² -0.559
Ep 1 (Step 000060): Train MAE 427.112, Val MAE 369.649, Train R² -0.472, Val R² -0.558
Ep 1 (Step 000070): Train MAE 379.262, Val MAE 369.651, Train R² -0.260, Val R² -0.558
Training MAE: 389.1299 | Validation MAE: 369.6357 | Training RMSE: 488.8282 | Validation RMSE: 465.1198 | Training R²: -0.2834 | Validation R²: -0.5583
Ep 2 (Step 000080): Train MAE 379.691, Val MAE 369.628, Train R² -0.212, Val R² -0.558
Ep 2 (Step 000090): Train MAE 363.690, Val MAE 369.614, Train R² 

KeyboardInterrupt: 

In [33]:
torch.save(model.state_dict(), "lstm-5-no-txt-grad.pt")
torch.save(optimizer.state_dict(), "optim.pt")

In [ ]:
# model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()